在正式开始训练之前，我们先用一个简单的示例跑一跑训练过程。下面的示例是一个复制内容的任务，给定一个词汇表中的随机字符串，目标是生成相同的字符串。

# 合成数据
下面是一个产生批次数据的函数，传入词表`V`，批次大小`batch_size`，批次数量`nbatched`，然后一个批次一个批次的生成数据，这里建议好好看了一下`yield`函数，这个函数不会一次返回所有批次的数据，而是一个批次一个批次的返回，只有当模型需要数据的时候，才会返回一个批次数据，这对于大规模预训练中，是必不可少的。

In [ ]:
"""
返回一个张量，其中充满在low（包含）和high（不包含）之间均匀生成的随机整数。 张量的形状由参数siz定义。

Parameters

-  **low**: 生成的随机整数的下界. Default: 0.
-  **high**： 生成的随机整数的上界.
-  **size**： 生成的随机整数的张量的形状，可以是一个整数或元组
    
Keyword Arguments
-   **generator**：一个用于采样的伪随机数生成器
-   **out**： 输出的tensor.
-   **dtype** (torch.dtype, optional)： 生成的数据类型，如果为None，是 `torch.int64`.
-   **layout**：返回张量的期望布局. Default: `torch.strided`.
-   **device**：返回张量的期望装置。Default：如果为None，则使用当前设备作为默认张量类型。设备将是CPU张量类型的CPU和当前CUDA张量类型的CUDA设备。
-   **requiresgrad**：If autograd should record operations on the returned tensor. Default: False.
    

>>> torch.randint(3, 10, (2, 2))
tensor([[4, 5],
        [6, 7]])

"""
torch.randint(low=0, high, size, \*, generator=None, out=None, dtype=None, layout=torch.strided, device=None, requiresgrad=False) → [Tensor]

## detach的作用
detach()函数在Pytorch中用于从当前计算图中分离张量，分离是创建一个新的张量，原始张量在计算图中依旧存在，并不受影响。但是新张量不参与梯度计算，与原始张量共享数据。因此，对分离后的张量进行的任何操作都不会影响原始张量，也不会在计算图中留下任何痕迹。

由于涉及到计算图，这个会在后续进行补充。

In [9]:
import torch
from utils.batch import Batch

def data_gen(V, batchsize, nbatches):
    "Generate random data for a src-tgt copy task."
    for i in range(nbatches):
        data = torch.randint(1, V, size=(batchsize, 10))
        data[:, 0] = 1
        src = data.requires_grad_(False).clone().detach() # detach()返回一个新的Variable，从当前计算图中分离下来的，但是仍指向原变量的存放位置，即如果原变量的数据发生了改变，新的Variable
        tgt = data.requires_grad_(False).clone().detach()
        yield Batch(src, tgt, 0)

# Loss计算
下面代码中的generator就是在前文中提到的Genrator类，用以在Decoder之后的线性变化和Softmax操作，最终输出词的预测概率。这里是把这个操作放到了loss计算中，当然也可以先计算再传入到loss计算里面。criterion就是损失计算函数，用的是前文中写的LabelSmoothing。

@QA
@Q 讲一下contiguous()函数的作用
contiguous()就相当于深拷贝操作，拷贝一份tensor，保证接下来的操作不会对原tensor造成影响。这里要注意的是在torch的，对tensor进行narrow()、view()、expand()和transpose()等操作都不会创建新的tensor，都是在原数据的基础上进行操作，也就是操作前后的tensor是共享内存的。

@QA
@Q 讲一下view()函数的作用
按照传入的参数变换维度，与转置不同，view操作会先按行将tensor展开成一维，然后按照传入参数的维度要求，去组成对应维度的tensor。看下面这个示例：
import torch
a=torch.Tensor([[[1,2,3],[4,5,6]]]) # torch.Size([1, 2, 3])
print(a.view(3,2))                  # torch.Size([3, 2])
输出：
tensor([[1., 2.],
        [3., 4.],
        [5., 6.]])

In [6]:
class SimpleLossCompute:
    def __init__(self, generator, criterion):
        self.generator = generator
        self.criterion = criterion

    def __call__(self, x, y, norm):
        x = self.generator(x)
        sloss = (
            self.criterion(
                x.contiguous().view(-1, x.size(-1)), y.contiguous().view(-1)
            )
            / norm
        )
        return sloss.data * norm, sloss

# 贪心解码
贪心解码策略就是每步都选取概率最高的词作为预测的词。代码如下：

In [7]:
from utils.utils import subsequent_mask

def greedy_decode(model, src, src_mask, max_len, start_symbol):
    memory = model.encode(src, src_mask)
    ys = torch.zeros(1, 1).fill_(start_symbol).type_as(src.data)
    for i in range(max_len - 1):
        out = model.decode(
            memory, src_mask, ys, subsequent_mask(ys.size(1)).type_as(src.data)
        )
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.data[0]
        ys = torch.cat(
            [ys, torch.zeros(1, 1).type_as(src.data).fill_(next_word)], dim=1
        )
    return ys

# 开始训练
下面开始训练，词表大小为11，EncodeLayer和DecoderLayer只有2层。

In [10]:
from conf.settings import DummyOptimizer, DummyScheduler
from Train import LabelSmoothing
from Train.learning_rate import rate
from torch.optim.lr_scheduler import LambdaLR
from Train.train import run_epoch
from EncoderDecoder import make_model

def example_simple_model():
    V = 11
    criterion = LabelSmoothing(size=V, padding_idx=0, smoothing=0.0)
    model = make_model(V, V, N=2)

    optimizer = torch.optim.Adam(
        model.parameters(), lr=0.5, betas=(0.9, 0.98), eps=1e-9
    )
    lr_scheduler = LambdaLR(
        optimizer=optimizer,
        lr_lambda=lambda step: rate(
            step, model_size=model.src_embed[0].d_model, factor=1.0, warmup=400
        ),
    )

    batch_size = 80
    for epoch in range(20):
        model.train()
        run_epoch(
            data_gen(V, batch_size, 20),
            model,
            SimpleLossCompute(model.generator, criterion),
            optimizer,
            lr_scheduler,
            mode="train",
        )
        model.eval()
        run_epoch(
            data_gen(V, batch_size, 5),
            model,
            SimpleLossCompute(model.generator, criterion),
            DummyOptimizer(),
            DummyScheduler(),
            mode="eval",
        )[0]

    model.eval()
    src = torch.LongTensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])
    max_len = src.shape[1]
    src_mask = torch.ones(1, 1, max_len)
    print(greedy_decode(model, src, src_mask, max_len=max_len, start_symbol=0))


example_simple_model()

Epoch Step:      1 | Accumulation Step:   2 | Loss:   3.12 | Tokens / Sec:  2189.2 | Learning Rate: 5.5e-06
Epoch Step:      1 | Accumulation Step:   2 | Loss:   2.06 | Tokens / Sec:  2487.9 | Learning Rate: 6.1e-05
Epoch Step:      1 | Accumulation Step:   2 | Loss:   1.83 | Tokens / Sec:  2549.1 | Learning Rate: 1.2e-04
Epoch Step:      1 | Accumulation Step:   2 | Loss:   1.48 | Tokens / Sec:  2891.8 | Learning Rate: 1.7e-04
Epoch Step:      1 | Accumulation Step:   2 | Loss:   1.11 | Tokens / Sec:  2678.7 | Learning Rate: 2.3e-04
Epoch Step:      1 | Accumulation Step:   2 | Loss:   0.66 | Tokens / Sec:  2765.7 | Learning Rate: 2.8e-04
Epoch Step:      1 | Accumulation Step:   2 | Loss:   0.34 | Tokens / Sec:  2566.6 | Learning Rate: 3.4e-04
Epoch Step:      1 | Accumulation Step:   2 | Loss:   0.17 | Tokens / Sec:  2775.3 | Learning Rate: 3.9e-04
Epoch Step:      1 | Accumulation Step:   2 | Loss:   0.14 | Tokens / Sec:  2683.9 | Learning Rate: 4.5e-04
Epoch Step:      1 | Accumul

<p align="center">
    <img src="imgs/train/train_example_res.png" width="100%">
</p>